In [1]:
import os

import io

import csv 

from zipfile import ZipFile

import glob 

import random 

from google.cloud import bigquery

from google.oauth2 import service_account

import sqlite3

In [2]:
# Rename "holder" folder 
zip_files = os.listdir('WedgeZips/')

In [3]:
# List of headers
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type",
           "trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total",
           "regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable",
           "discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch",
           "matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID",
           "local","organic","display","receipt","card_no","store","branch","match_id","trans_id"]

In [4]:
# Make a new directory 
temp_folder_name = "wedge_clean/"

if not os.path.isdir(temp_folder_name) :
    os.mkdir(temp_folder_name)

In [5]:
# Unzipping files, clean the files, add them to zip_clean folder 

# For zip file in zipfiles, read the file in the directory and assign a new name for the files inside
for zip_file in zip_files : 
    print(zip_file)
    
    with ZipFile("WedgeZips/" + zip_file, 'r') as my_zip_file :
        files_inside = my_zip_file.namelist()
        
        for zipped_file in files_inside :
            sniffer = csv.Sniffer()
            
            # Open and read input files then rename new files
            with my_zip_file.open(zipped_file,'r') as input_file :
                clean_file_name = input_file.name.replace('.csv','_clean.csv') 
                
                # Open new output file names in new folder, write files and join headers to new outfile
                with open("wedge_clean/" + clean_file_name,'w') as outfile :
                    outfile.write(",".join(headers) + "\n")
                    rows_printed = 0
                    
                    # Begin cleaning the files
                    # Bytes object line to a normal string - strip and split string to list
                    # List comprehension to remove the double quotes
                    # Remove all null vaules
                    for idx,line in enumerate(input_file):
                        file_has_header = False
                        dialect = sniffer.sniff(line.decode("utf-8"))
                        line = line.decode("utf-8").strip().split(dialect.delimiter)
                        line=[piece.replace(r'"','') for piece in line] 
                        line=[piece.replace(r"//N",'') for piece in line]
                        line=[piece.replace(r"\N",'') for piece in line]
                        line=[piece.replace(r'NULL','') for piece in line]
                        
                        # Make new column and get rid of the bad!
                        if len(line) != 50 :
                            new_column = ("".join(line[5:8])) 
                        
                            del(line[5:8])
                        
                            line.insert(5,new_column)
                        
                        # For the first line (if header), do not print, otherwise, outwrite file
                        if idx == 0:
                            if 'datetime' in line[0]:
                                file_has_header = True
                        if file_has_header and idx == 0:
                            pass #don't print the line
                        else:
                            outfile.write(",".join(line) + "\n")
                            rows_printed += 1
print(f"Zip-A-Dee-Doo-Da")

transArchive_201410_201412.zip
transArchive_201301_201303_inactive.zip
transArchive_201210_201212.zip
transArchive_201609.zip
transArchive_201608.zip
transArchive_201201_201203.zip
transArchive_201204_201206.zip
transArchive_201407_201409.zip
transArchive_201207_201209.zip
transArchive_201404_201406.zip
transArchive_201401_201403.zip
transArchive_201404_201406_inactive.zip
transArchive_201210_201212_inactive.zip
transArchive_201307_201309_inactive.zip
transArchive_201501_201503.zip
transArchive_201307_201309.zip
transArchive_201504_201506.zip
transArchive_201304_201306.zip
transArchive_201507_201509.zip
transArchive_201301_201303.zip
transArchive_201204_201206_inactive.zip
transArchive_201410_201412_inactive.zip
transArchive_201310_201312.zip
transArchive_201106.zip
transArchive_201110_201112.zip
transArchive_201701.zip
transArchive_201310_201312_inactive.zip
transArchive_201105.zip
transArchive_201104.zip
transArchive_201510.zip
transArchive_201201_201203_inactive.zip
transArchive_201

In [ ]:
from google.cloud import bigquery 
from google.oauth2 import service_account
import os
import csv

In [ ]:
file_path = "/Users/clairedanicich/Desktop/ADA/wedge/wedge2020/wedge_clean/"

In [ ]:
clean_wedge = os.listdir("wedge_clean/")

In [ ]:
# Authentication: these keys will be different on your device, must change to run your own code. 
service_path = "/Users/clairedanicich/Desktop/ADA/wedge/wedge2020/"
service_file = "MSBA-cfc85a1b18d5.json"
gbq_proj_id = "msba-291619"
gbq_dataset_id = "wedge2"

privatekey = service_path + service_file

# Pass in credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# Establish connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
# Test to see if table exists, if no table, create one
def tbl_exists(client, table_ref) :
    from google.cloud.exceptions import NotFound
    try :
        client.get_table(table_ref)
        return True 
    except NotFound :
        return False

In [ ]:
# create new table names by running through the files then uploading to GBQ
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job_config.schema_update_options = [
    # This allows us to modify the table. 
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION 
]

In [ ]:
job_config.schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
]
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1

In [ ]:
for file in clean_wedge :
    #splits on _clean in csv files and removes rest as junk
    my_table,junk = file.split("_clean")

    #creates GBQ table name
    table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])


    if not tbl_exists(client, table_full_name) :
        table_ref = client.create_table(table = table_full_name)
    else :
        table_ref = client.get_table(table_full_name)
    
    table = client.get_table(table_ref)
    print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))
    
    with open(file_path + file, "rb") as source_file:
        job = client.load_table_from_file(
            source_file,
            table_ref,
            location="US",  # Must match the destination dataset location.
            job_config=job_config
        )  # API request

    # Waits for table load to complete.
    job.result()  
    print("Loaded {} rows into {}:{}.".format(job.output_rows, 'wedge_example', table_ref.table_id))
    
    
    # Checks the updated length of the schema
    table = client.get_table(table)
    print("Table {} now contains {} columns.".format(table_ref.table_id, len(table.schema)))